In [1]:
# Path of your file
nom_fichier = "final1"

In [5]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
def showtree(a):
    pos = nx.spring_layout(a) 
    nx.draw(a, pos, with_labels=True, node_color='lightblue', node_size=700, font_size=12, font_weight='bold')

In [6]:
#initialization of all the dict
ip = {}
link = {}
pr={}
id = {}
com = {}
temp = {}
bat = {}
comp = 0

In [ ]:
with open(nom_fichier, "r") as fichier:
  lignes = fichier.readlines()
  for l in lignes:
        if (len(l)>20):
            r = l[20:].index(';')
            a = l[21:21+r-1]
            if (a not in ip) :
              ip[a] = 0
              link[a] = 0
              pr[a] = []
              id[a] = 0
              com[a] = 0
              bat[a] = [100]

In [ ]:
with open(nom_fichier, "r") as fichier:
  lignes = fichier.readlines()
  for l in lignes:
    if (l[-10:]=='link  VAL\n'):
      a = l.index(':')
      r = l[20:].index(';')
      i = l[21:21+r-1]
      val = l[a+2:a+27]
      ip[i]=val.replace(" ", "")
    if ('*' in l):
      val = l[l.index('v')+4:l.index('v')+29]
      r = l[20:].index(';')
      i = l[21:21+r-1]
      link[i]=val.replace(" ","")
    if ('parent' in l and 'addr' in l):
      a = l[l.index('[')+7:l.index('[')+32]
      r = l[20:].index(';')
      i = l[21:21+r-1]
      pr[i].append(a.replace(" ",""))
    if 'ID =' in l[20:] :
      r = l[20:].index(';')
      a = l[21:21+r-1]
      id[a] = l[l.index('=')+2:].replace("\n","")
      temp[l[l.index('=')+2:].replace("\n","")]=0
    if 'Packet' in l:
      sup = l[21:21+l[20:].index(';')-1]
      r = l[l.index(':')+2:].replace("\n","")
      r = r[:3].replace(" ","")
      if temp[r]!=l[:10]:
        for i in id :
              if r == id[i]:
                com[i] = com[i] + 1
      temp[r] = l[:10]
    if "changement" in l:
      comp = comp + 1
    r = l[20:].index(';')
    a = l[21:21+r-1]
    if (l[21+r:24+r]=='bat'):
      res = l[-4:].replace("\n","")
      res = res.replace(" ","")
      res = res.replace("=", "")
      bat[a].append(float(res)/10)

In [ ]:
G = nx.Graph()
for i in ip :
  G.add_node(i)
for i in link:
  a =link[i]
  for j in ip:
    if (a == ip[j]):
      G.add_edge(j, i)
    
H = nx.Graph()
for i in ip :
  H.add_node(i)
for i in pr:
  for k in range(len(pr[i])):
    a = pr[i][k]
    for j in ip:
      if (a == ip[j]):
        H.add_edge(j, i)

In [ ]:
showtree(H)

In [ ]:
showtree(G)

In [ ]:
del ip[sup]
del com[sup]

start = ""

with open(nom_fichier, "r") as fichier:
  lignes = fichier.readlines()
  for l in lignes:
        if 'Packet' in l :
          start = l[:10]
          break

with open(nom_fichier, "r") as fichier:
  lignes = fichier.readlines()
  for l in lignes:
        if 'Packet' in l :
          end = l[:10]


time = (int(end)-int(start))
nb = time/4

p = []
q = []
lab = []
for i in com :
  q.append(com[i])
  p.append((com[i]/nb)*100)
  lab.append(i)

for i in range(len(p)):
  if p[i]>100 :
    p[i]=97

plt.yticks(np.arange(0, 105, 5))

plt.bar(lab, p)

plt.xlabel('Sensors')
plt.ylabel('% of packets received correctly')
plt.title('Percentage of packets received correctly per sensor')

plt.show()

In [ ]:
def moy(liste):
    somme = sum(liste)
    moyenne = somme / len(liste)
    return moyenne

print(f"packets lost in the network {100-moy(p)}")

In [ ]:
for cle, tableau in bat.items():
    plt.plot(tableau, label=cle)


plt.yticks(np.arange(0, 105, 5))
plt.legend(loc='upper left')
plt.xlabel('time in minutes')
plt.ylabel('% of battery')
plt.title('Percentage battery of each sensor as a function of time ')
plt.show()

In [ ]:
print(f"the number of route changes is {comp}")